###Problem Statement

A real estate company that has a dataset containing the prices of properties in the Delhi region. It wishes to use the data to optimise the sale prices


This company specifically considering area of the property and prefered area to choose
However, they are open to the possibility that other variables may also affect the price of a home

The data source that I'm using in this Analysis is from Kaggle : [Housing Price Prediction](https://www.kaggle.com/code/ashydv/housing-price-prediction-linear-regression)




#Data Overview

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#Import library

#Load data
import pandas as pd
import numpy as np

#Visualization
import matplotlib.pyplot as plt
import seaborn as sns

#Modelling
import statsmodels.formula.api as smf

#Statistics
import scipy.stats as stats

In [ ]:
housing = pd.read_csv("/content/drive/MyDrive/Pacmann/Project/Python Project/Linear Regression Project - Housing.csv")
housing.head(10)

In [ ]:
#Get the information of rows & columns
housing.shape

- The data consists of 545 rows & 13 columns



In [ ]:
#Get the datatype in each columns
housing.info()

- From the info above, some data are considered as object. However, those data are stored in a categorical format. Therefore I would like to encode those data to numerical value in order to be processed

In [ ]:
#Checking the number of unique object in each variables
x = ["mainroad","guestroom", "basement", "hotwaterheating", "airconditioning", "prefarea","furnishingstatus"]

for column in x:
  unique_values = housing[column].unique()
  print(f"Unique values in '{column}':{unique_values}")

In [ ]:
#Label encoding categorical data to numerical value
housing['mainroad'] = housing['mainroad'].map({'yes':1, 'no':0})
housing['guestroom'] = housing['guestroom'].map({'yes':1, 'no':0})
housing['basement'] = housing['basement'].map({'yes':1, 'no':0})
housing['hotwaterheating'] = housing['hotwaterheating'].map({'yes':1, 'no':0})
housing['airconditioning'] = housing['airconditioning'].map({'yes':1, 'no':0})
housing['prefarea'] = housing['prefarea'].map({'yes':1, 'no':0})
housing['furnishingstatus'] = housing['furnishingstatus'].map({'unfurnished':1, 'semi-furnished':2, 'furnished':3})

In [ ]:
#Re-checking the data information
housing.info()

- all of the string datatype variables have turned into int64

In [ ]:
#Missing Value
missing_value = housing.isnull().sum()
print(missing_value)

In [ ]:
#Outlier checking for numerical data
fig, axs = plt.subplots(2, 3, figsize = (10,6))

sns.boxplot(housing['price'], ax=axs[0, 0]).set_title('Price')
sns.boxplot(housing['area'], ax=axs[0, 1]).set_title('Area')
sns.boxplot(housing['bedrooms'], ax=axs[0, 2]).set_title('Bedrooms')
sns.boxplot(housing['bathrooms'], ax=axs[1, 0]).set_title('Bathrooms')
sns.boxplot(housing['stories'], ax=axs[1, 1]).set_title('Stories')
sns.boxplot(housing['parking'], ax=axs[1, 2]).set_title('Parking')

plt.tight_layout()
plt.show()

- Quite a large amount of outlier in 'Price' and 'Area' metrics
- However, other metrics have the potential of outliers due to each house property diversities, which make sense
- Next, I'd like to cut off the outliers of 'Price' & 'Area' from the data

In [ ]:
#Cutting off Price column's outlier
Q1 = housing['price'].quantile(0.25)
Q3 = housing['price'].quantile(0.75)
IQR = Q3-Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR
#housing = housing[(housing['price']>= lower_bound) & (housing['price']<= upper_bound)]

#Cutting off Area column's outlier
Q1 = housing['area'].quantile(0.25)
Q3 = housing['area'].quantile(0.75)
IQR = Q3-Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR
#housing = housing[(housing['area']>= lower_bound) & (housing['area']<= upper_bound)]

#Visualization

In [ ]:
#Correlation checking
plt.figure(figsize=(10, 8))

col = housing.columns
heatmap = sns.heatmap(housing[col].corr(), vmin=-1, vmax=1, annot=True, cmap='BrBG')

heatmap.set_title('Correlation Heatmap', fontdict={'fontsize':18}, pad=12);

- By looking at the correlation, the variables that have quite high correlation with price are area and bathrooms
- The other variables have lower than |0.5| correlation with price
- Price and bathroom might significantly affecting the price, however we still want to know which variables create the best model

In [ ]:
#Visualization between all variables & price
columns = ['area', 'bedrooms', 'bathrooms', 'stories', 'mainroad', 'guestroom',
       'basement', 'hotwaterheating', 'airconditioning', 'parking', 'prefarea',
       'furnishingstatus']

fig, axes = plt.subplots(nrows=2, ncols=6, figsize=(14, 6))
for i, ax in zip(columns, axes.flatten()):
  sns.scatterplot(y = "price", x = i, data = housing, ax=ax)
  ax.set_title(f"House price & {i}")

plt.tight_layout()

In [ ]:
#Visualization boxplot between all variables & price (to see the median)
columns = ['area', 'bedrooms', 'bathrooms', 'stories', 'mainroad', 'guestroom',
       'basement', 'hotwaterheating', 'airconditioning', 'parking', 'prefarea',
       'furnishingstatus']

fig, axes = plt.subplots(nrows=2, ncols=6, figsize=(14, 5))
for i, ax in zip(columns, axes.flatten()):
  sns.boxplot(y = "price", x = i, data = housing, ax=ax)
  ax.set_title(f"House price & {i}")

plt.tight_layout()

- all variables tend to have a positive relation, which shown the higher the variables value, the house price will also become higher

#Create Model

- Wanted to create a regression model with area and prefarea

###Model 1

In [ ]:
def print_coef_std_err(results):
  coef = results.params
  std_err = results.bse

  df = pd.DataFrame(data = np.transpose([coef, std_err]),
                    index = coef.index,
                    columns = ["coef", "std err"])

  return df

In [ ]:
#Create OLS model object
model = smf.ols("price ~ area", housing)

#Fit model
results = model.fit()

# Extract the results (Coefficient and Standard Error) to DataFrame
coff_stderr = print_coef_std_err(results)
coff_stderr

$$\text{price} = 2,387,308 + 461.97\text{area}$$
---

$$\text{price} = 2,387,308 + 461.97\text{area}$$
---



- If the area is 0, then the price will be 2.3mio
- If there is an increment in the area by 1, then there will be additional 461.97 in the price

In [ ]:
area = 100
price = 2387308 + 461.97*area
price

In [ ]:
area = 200
price = 2387308 + 461.97*area
price

####Plot Data & Estimated Line

In [ ]:
#Buat visualization untuk line estimation
predictor = 'area'
outcome = 'price'
data = housing.copy()
results_ = coff_stderr.copy()

#Buat visualization antara price & area
plt.scatter(housing['area'], housing['price'], color = 'orange')

a_hat = results_.loc['Intercept']['coef']
b_hat = results_.loc[predictor]['coef']
x_domain = np.linspace(np.min(data[predictor]), np.max(data[predictor]), 1000)
y_hat = a_hat + b_hat*x_domain
plt.plot(x_domain, y_hat, color = 'black')

plt.xlabel('Price')
plt.ylabel('Area')
plt.title('Price vs Area')

- Because the data appears to exhibit increasing spread as the price increases, I want to check the assumption of homoscedasticity for the residuals
- Does it meet the assumption of homoscedasticity?

In [ ]:
plt.scatter(results.fittedvalues, results.resid, marker=".", c = "k")

# Plot the horizontal line in 0 as the fitted line
plt.axhline([0])

plt.xlabel("predicted price")
plt.ylabel("residual")
plt.title(f"Residuals vs. Predicted price")

- The residuals are not random
- Tend to have a pattern

In [ ]:
###Normality of Error Assumption

In [ ]:
plt.hist(results.resid, color='tab:blue', alpha=0.4)
plt.xlabel("residual")
plt.ylabel("count")

plt.show()

- The residuals are skewed to the left, thus not normal

###Model 2 - Weighted Least Square

In [ ]:
weights_area = 1/(housing['area']**2)

In [ ]:
housing['weights_area']= weights_area

In [ ]:
housing.sort_values(by = 'price')

In [ ]:
#Create OLS model object
model = smf.ols("price ~ weights_area", housing)

#Fit model
results = model.fit()

# Extract the results (Coefficient and Standard Error) to DataFrame
coff_stderr = print_coef_std_err(results)
coff_stderr

In [ ]:
#Buat visualization untuk line estimation
predictor = 'weights_area'
outcome = 'price'
data = housing.copy()
results_ = coff_stderr.copy()

#Buat visualization antara price & area
plt.scatter(housing['weights_area'], housing['price'], color = 'orange')

a_hat = results_.loc['Intercept']['coef']
b_hat = results_.loc[predictor]['coef']
x_domain = np.linspace(np.min(data[predictor]), np.max(data[predictor]), 1000)
y_hat = a_hat + b_hat*x_domain
plt.plot(x_domain, y_hat, color = 'black')

plt.xlabel('Weighted Area')
plt.ylabel('Price')
plt.title('Price vs Area')

In [ ]:
results.summary()

- The accuracy is very low

###Model 3 - Log Transform

In [ ]:
housing['log_area'] = np.log(housing["area"])
housing.head(3)

In [ ]:
#Create OLS model object
model = smf.ols("price ~ log_area", housing)

#Fit model
results = model.fit()

# Extract the results (Coefficient and Standard Error) to DataFrame
coff_stderr = print_coef_std_err(results)
coff_stderr

In [ ]:
#Buat visualization untuk line estimation
predictor = 'log_area'
outcome = 'price'
data = housing.copy()
results_ = coff_stderr.copy()

#Buat visualization antara price & area
plt.scatter(housing['log_area'], housing['price'], color = 'orange')

a_hat = results_.loc['Intercept']['coef']
b_hat = results_.loc[predictor]['coef']
x_domain = np.linspace(np.min(data[predictor]), np.max(data[predictor]), 1000)
y_hat = a_hat + b_hat*x_domain
plt.plot(x_domain, y_hat, color = 'black')

plt.xlabel('Log Area')
plt.ylabel('Price')
plt.title('Price vs Area')

In [ ]:
results.summary()

In [ ]:
#Adding furnishing to the model
#Create OLS model object
model = smf.ols("price ~ log_area + furnishingstatus", housing)

#Fit model
results_1 = model.fit()

# Extract the results (Coefficient and Standard Error) to DataFrame
coef_stderr_1 = print_coef_std_err(results_1)
coef_stderr_1

In [ ]:
results_1.summary()

###Model 4 - OLS with All Variables

In [ ]:
#Create OLS model object
model_all = smf.ols("price ~  area + bedrooms + bathrooms + stories + \
mainroad + guestroom + basement + hotwaterheating + airconditioning + parking + furnishingstatus", housing)

#Fit model
results_all = model_all.fit()
results_all.summary()

- eliminate the variables that have p-value > 0.05

In [ ]:
#Create OLS model object
model_all = smf.ols("price ~  area + bathrooms + stories + \
mainroad + basement + hotwaterheating + airconditioning + parking + furnishingstatus", housing)

#Fit model
results_all = model_all.fit()
results_all.summary()